In [1]:
import sys
sys.path.append("../")

import pandas as pd
import os
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import print_function
from patsy import dmatrices
from openpyxl import load_workbook

from library import regulations
from library import characteristics
from library import analysis
from library import tables

In [2]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits.csv'),
                  sep=",", low_memory = False)
data.head()
data = data[data.year == 2016]
data['doi_16'] = np.where(data.doi_year == 2016, 1, 0)
data['doi_17'] = np.where(data.doi_year == 2017, 1, 0)
data['doi_18'] = np.where(data.doi_year == 2018, 1, 0)
data['tpsd'] = np.where(data.doi == False, 1, 0)
data.head()

,Unnamed: 0,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,...,treatyear,treatpost,treatpostyear,yearpost1,treatpostyear1,treatpostyear2,doi_16,doi_17,doi_18,tpsd
3244,4909,1902,CAYUGA ISD,2016,ANDERSON,N,M,Pass,True,H,...,-1.0,0,-0.0,0,0,0,0,1,0,0
3245,4910,1903,ELKHART ISD,2016,ANDERSON,N,M,Pass,True,G,...,-2.0,0,-0.0,0,0,0,0,0,1,0
3246,4911,1904,FRANKSTON ISD,2016,ANDERSON,N,M,Pass,True,H,...,-2.0,0,-0.0,0,0,0,0,0,1,0
3247,4912,1906,NECHES ISD,2016,ANDERSON,N,M,Pass,True,H,...,-1.0,0,-0.0,0,0,0,0,1,0,0
3248,4913,1907,PALESTINE ISD,2016,ANDERSON,N,M,Pass,True,E,...,0.0,1,0.0,0,0,0,1,0,0,0


# Characteristics by Adoption Year

In [3]:
y, X = dmatrices('students_num ~ doi + doi_16 + doi_18', data=data, return_type='dataframe', NA_action = "drop")
X.head()

,Intercept,doi[T.True],doi_16,doi_18
3244,1.0,1.0,0.0,0.0
3245,1.0,1.0,0.0,1.0
3246,1.0,1.0,0.0,1.0
3247,1.0,1.0,0.0,0.0
3248,1.0,1.0,1.0,0.0


In [4]:
mod = sm.OLS(y, X)    # Describe model
res = mod.fit()       # Fit model
print(res.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:           students_num   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     4.936
Date:                Wed, 10 Apr 2019   Prob (F-statistic):            0.00211
Time:                        10:18:05   Log-Likelihood:                -8964.8
No. Observations:                 812   AIC:                         1.794e+04
Df Residuals:                     808   BIC:                         1.796e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    6782.1863   1191.779      5.691      

# Make mini table

In [5]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)
test

'0.1**'

In [6]:
test = ['1', '2']

In [7]:
coef_1 = []
formula = 'math' + ' ~ ' + 'doi + doi_year'
y, X = dmatrices('students_num ~ doi + doi_year', data=data, return_type='dataframe', NA_action = "drop") 
mod = sm.OLS(y, X)    # Describe model
res = mod.fit()       # Fit model
print(res.summary())   # Summarize model
result = res.summary()
res.params['doi[T.True]']
res.bse['doi_year']
#coef_1.append(coef_with_stars(res.params['doi']))

                            OLS Regression Results                            
Dep. Variable:           students_num   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     5.846
Date:                Wed, 10 Apr 2019   Prob (F-statistic):             0.0159
Time:                        10:18:05   Log-Likelihood:                -7091.4
No. Observations:                 651   AIC:                         1.419e+04
Df Residuals:                     649   BIC:                         1.420e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    1.995e+06   8.24e+05      2.421      

817.1638327987297

In [8]:
def make_coef_df(data, y_list, y_labels):
    regs = []
    cons = []
    coef_1 = []
    coef_2 = []
    coef_3 = []

    for var in y_list:
        formula = var + ' ~ ' + 'doi + doi_16 + doi_18'
        y, X = dmatrices(formula, data=data, return_type='dataframe', NA_action = "drop") 
        mod = sm.OLS(y, X)    # Describe model
        res = mod.fit()       # Fit model
        print(y_labels[var])
        print(res.summary())   # Summarize model
        cons.append(str(res.params["Intercept"].round(2)))
        cons.append('')
        coef_1.append(coef_with_stars(res.params['doi[T.True]'].round(2), res.pvalues['doi[T.True]']))
        coef_1.append('(' + str(res.bse['doi[T.True]'].round(2)) + ')')
        coef_2.append(coef_with_stars(res.params['doi_16'].round(2), res.pvalues['doi_16']))
        coef_2.append('(' + str(res.bse['doi_16'].round(2)) + ')')
        coef_3.append(coef_with_stars(res.params['doi_18'].round(2), res.pvalues['doi_18']))
        coef_3.append('(' + str(res.bse['doi_18'].round(2)) + ')')
        regs.append(y_labels[var])
        regs.append('')


    df = pd.DataFrame(
        {'Characteristic': regs,
         'Control': cons,
         'DOI Difference': coef_1,
         'Early Adopter': coef_2,
         'Late Adopter': coef_3,
         })
    return df
geo = make_coef_df(data = data, 
                                 y_list = characteristics.geography,
                                 y_labels = characteristics.labels)
geo

Urban
                            OLS Regression Results                            
Dep. Variable:             type_urban   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.488
Date:                Wed, 10 Apr 2019   Prob (F-statistic):              0.216
Time:                        10:18:05   Log-Likelihood:                 56.294
No. Observations:                 812   AIC:                            -104.6
Df Residuals:                     808   BIC:                            -85.79
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0807      0.018      4.527

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Urban,0.08,-0.03,0.03,-0.01
1,,,(0.02),(0.03),(0.02)
2,Suburban,0.25,-0.03,0.2***,0.05
3,,,(0.04),(0.06),(0.04)
4,Town,0.26,0.04,-0.01,-0.02
5,,,(0.04),(0.06),(0.05)
6,Rural,0.41,0.03,-0.22**,-0.01
7,,,(0.04),(0.07),(0.05)


In [9]:
teacher = make_coef_df(data = data, 
                                 y_list = characteristics.teacher,
                                 y_labels = characteristics.labels)
teacher

Ave. Experience Teaching
                             OLS Regression Results                            
Dep. Variable:     teachers_tenure_ave   R-squared:                       0.001
Model:                             OLS   Adj. R-squared:                 -0.003
Method:                  Least Squares   F-statistic:                    0.3243
Date:                 Wed, 10 Apr 2019   Prob (F-statistic):              0.808
Time:                         10:18:06   Log-Likelihood:                -1653.3
No. Observations:                  812   AIC:                             3315.
Df Residuals:                      808   BIC:                             3333.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Ave. Experience Teaching,7.39,-0.12,-0.09,-0.06
1,,,(0.17),(0.25),(0.19)
2,Teacher Turnover Ratio,0.19,-0.0,-0.02,-0.01
3,,,(0.01),(0.01),(0.01)
4,Student-Teacher Ratio,6.41,0.2,0.51**,0.25*
5,,,(0.11),(0.15),(0.12)


In [10]:
student = make_coef_df(data = data, 
                                 y_list = characteristics.student,
                                 y_labels = characteristics.labels)
student

Percent Hispanic
                            OLS Regression Results                            
Dep. Variable:          students_hisp   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     17.24
Date:                Wed, 10 Apr 2019   Prob (F-statistic):           7.35e-11
Time:                        10:18:06   Log-Likelihood:                -64.267
No. Observations:                 812   AIC:                             136.5
Df Residuals:                     808   BIC:                             155.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.5221      0.021

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Percent Hispanic,0.52,-0.17***,0.07,0.02
1,,,(0.02),(0.04),(0.03)
2,Percent White,0.4,0.14***,-0.09**,-0.01
3,,,(0.02),(0.03),(0.03)
4,Percent Black,0.05,0.02*,0.0,-0.01
5,,,(0.01),(0.01),(0.01)
6,Percent Econ. Disadvantaged,0.62,-0.07***,-0.03,-0.01
7,,,(0.02),(0.02),(0.02)
8,Average STAAR Performance (Std.),0.17,0.22***,0.25**,0.02
9,,,(0.06),(0.09),(0.07)


# To Table

In [11]:
rows = [6, 15, 22]

In [12]:
dfs = [geo, teacher, student]
file = table_path + 'regression_doi_year.xlsx'
wb = load_workbook(file)
ws = wb.active
str_n = 'N = ' + str(len(data[data.doi == False]))
ws.cell(row=4, column=2).value = str_n
str_n = 'N = ' + str(len(data[data.doi == True]))

for df, row in zip(dfs, rows):
    ws.cell(row=4, column=3).value = str_n
    row_n = row
    for ob in df.Control:
        ws.cell(row=row_n, column=2).value = ob
        row_n = row_n + 1
    row_n = row
    for ob in df['DOI Difference']:
        ws.cell(row=row_n, column=3).value = ob
        row_n = row_n + 1   
    row_n = row
    for ob in df['Early Adopter']:
        ws.cell(row=row_n, column=4).value = ob
        row_n = row_n + 1 
    row_n = row
    for ob in df['Late Adopter']:
        ws.cell(row=row_n, column=5).value = ob
        row_n = row_n + 1 
wb.save(file)